In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from relation_templates.templates import relations, nominalized_relations

In [3]:
from TypologyQuerier import TypologyQuerier 

In [4]:
logging.set_verbosity_error()

In [5]:
model = "google/electra-base-generator"

In [6]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]

In [7]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
#relations = nominalized_relations
#relations = ['P138', 'P276', 'P31', 'P106', 'P413', 'P937', 'P1303', 'P127', 'P279', 'P527', 'P136', 'P39', 'P101', 'P361', 'P463', 'P176']
#relations =['P138', 'P276', 'P31', 'P937', 'P279', 'P527', 'P101', 'P361']

In [8]:
TOP_K = 1

In [9]:
#unmasker = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking', top_k=100)
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=100)
#unmasker = pipeline("fill-mask", model="google/electra-large-generator", tokenizer="google/electra-large-generator", top_k=TOP_K) 

In [10]:
TREx = LamaTRExData(relations = relations)
TREx.load()
#TREx.load_abstract()

In [11]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)

In [12]:
#querier.query(TREx.abstract_data)
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 25640.94it/s]


In [13]:
%%time
unmasker.save_to_cache()

CPU times: user 2 µs, sys: 6 µs, total: 8 µs
Wall time: 8.82 µs


In [14]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       609 |   34039 |   1.78912  |
| compound         |       256 |   34039 |   0.752078 |
| complex          |       117 |   34039 |   0.343723 |
| compound-complex |        58 |   34039 |   0.170393 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       457 |   20928 |   2.18368  |
| compound         |       189 |   20928 |   0.903096 |
| complex          |        96 |   20928 |   0.45

In [15]:
querier.print_global_latex()

Total  & 1.79 & 0.75 & 0.34 & 0.17 \\
1:1  & 0.00 & 0.00 & 0.00 & 0.00 \\
N:1  & 2.18 & 0.90 & 0.46 & 0.14 \\
N:M  & 1.25 & 0.55 & 0.17 & 0.24 \\


In [16]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P36 & capital & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P103 & native language & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P127 & owned by & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P131 & located in the administrative territorial entity & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P136 & genre & N:1 & 0.00 & 1.50 & 5.80 & 1.50 \\
P138 & named after & N:1 & 0.16 & 0.00 & 0.00 & 0.47 \\
P140 & religion or worldview & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P159 & headquarters location & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P17 & country & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P176 & manufacturer & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P19 & place of birth & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P20 & place of death & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P264 & record label & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P276 & location & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P279 & subclass of & N:1 & 23.03 & 17.74 & 2.59 & 0.41 \\
P30 & continent & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P31 

In [17]:
TOP_K = 10

In [18]:
querier.reset()
querier.set_top_k(TOP_K)

In [19]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 27461.64it/s]


In [20]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      1856 |   34039 |    5.45257 |
| compound         |      1326 |   34039 |    3.89553 |
| complex          |      1064 |   34039 |    3.12583 |
| compound-complex |      1348 |   34039 |    3.96016 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      1237 |   20928 |    5.91074 |
| compound         |       739 |   20928 |    3.53115 |
| complex          |       590 |   20928 |    2.8

In [21]:
querier.print_global_latex()

Total  & 5.45 & 3.90 & 3.13 & 3.96 \\
1:1  & 0.00 & 0.00 & 0.00 & 0.00 \\
N:1  & 5.91 & 3.53 & 2.82 & 3.69 \\
N:M  & 5.08 & 4.82 & 3.89 & 4.72 \\


In [22]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P36 & capital & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P103 & native language & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P127 & owned by & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P131 & located in the administrative territorial entity & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P136 & genre & N:1 & 28.79 & 26.32 & 35.12 & 17.29 \\
P138 & named after & N:1 & 0.31 & 0.31 & 0.16 & 0.93 \\
P140 & religion or worldview & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P159 & headquarters location & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P17 & country & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P176 & manufacturer & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P19 & place of birth & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P20 & place of death & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P264 & record label & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P276 & location & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P279 & subclass of & N:1 & 48.55 & 48.44 & 10.27 & 32.37 \\
P30 & continent & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \

In [23]:
TOP_K = 100

In [24]:
querier.reset()
querier.set_top_k(TOP_K)

In [25]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 12101.75it/s]


In [26]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      4253 |   34039 |   12.4945  |
| compound         |      3151 |   34039 |    9.25703 |
| complex          |      2739 |   34039 |    8.04665 |
| compound-complex |      3172 |   34039 |    9.31872 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      2233 |   20928 |   10.6699  |
| compound         |      1487 |   20928 |    7.10531 |
| complex          |      1458 |   20928 |    6.9

In [27]:
querier.print_global_latex()

Total  & 12.49 & 9.26 & 8.05 & 9.32 \\
1:1  & 0.00 & 0.00 & 0.00 & 0.00 \\
N:1  & 10.67 & 7.11 & 6.97 & 8.55 \\
N:M  & 16.59 & 13.67 & 10.52 & 11.36 \\


In [28]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P36 & capital & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P103 & native language & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P127 & owned by & N:1 & 0.15 & 0.00 & 0.15 & 0.15 \\
P131 & located in the administrative territorial entity & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P136 & genre & N:1 & 75.08 & 72.72 & 82.81 & 63.80 \\
P138 & named after & N:1 & 2.02 & 1.40 & 0.78 & 1.55 \\
P140 & religion or worldview & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P159 & headquarters location & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P17 & country & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P176 & manufacturer & N:1 & 0.00 & 0.10 & 0.00 & 0.10 \\
P19 & place of birth & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P20 & place of death & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P264 & record label & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P276 & location & N:1 & 0.00 & 0.00 & 0.10 & 0.00 \\
P279 & subclass of & N:1 & 66.29 & 65.66 & 29.15 & 56.22 \\
P30 & continent & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \